In [43]:
import os
from pathlib import Path
import re
import pandas as pd
from datetime import datetime

In [35]:
cifs_loc = "../Cubic CIFs/cifs/"
# cif_dataset_loc = "cifs"
felix_input_dataset_loc = "../Felix Input for AI Diffraction/input_data/"

In [3]:
# Path(cif_dataset_loc).mkdir(parents=True, exist_ok=True)
Path(felix_input_dataset_loc).mkdir(parents=True, exist_ok=True)

In [62]:
keyinfo = []

hkl_file = open("hkl_template.txt", "r", encoding="utf8")
hkl_content = hkl_file.read()

inp_file = open("inp_template.txt", "r", encoding="utf8")
inp_content = inp_file.read()

uniques = dict()

for cif_filename in os.listdir(cifs_loc):
    ICSD_code = int(cif_filename.split(".")[0])
    cif_file = open(os.path.join(cifs_loc, cif_filename), "r", encoding="utf8")
    cif_text = cif_file.read()
    cif_file.close()
    
    try:
        chemical_formula_sum = re.search(
            r"_chemical_formula_sum\s*([\s\S]*?)\s*_",
            cif_text).group(1).strip(r" \n;'").replace('\n', '')
    except:
        continue
    
    try:
        cell_length = float(re.search(r"_cell_length_a\s+([\d.]+)", cif_text).group(1))
    except:
        continue
    
    try:
        space_group = re.search(r"_space_group_name_H-M_alt '(.+)'", cif_text).group(1)
    except:
        continue
    
    try:
        date_str = re.search(r"_audit_creation_date (.+)", cif_text).group(1)
        date_obj = datetime.strptime(date_str, r"%Y-%m-%d")
    except:
        continue

    # if chemical_formula_sum not in uniques:
    #     uniques[chemical_formula_sum] = [date_obj, ICSD_code]
    # elif uniques[chemical_formula_sum][0] < date_obj:
    #     uniques[chemical_formula_sum] = [date_obj, ICSD_code]
    
    keyinfo.append(
        {
            "ICSD_code": ICSD_code,
            'chemical_formula_sum': chemical_formula_sum,
            'cell_length': cell_length,
            'space_group': space_group,
            'date': date_str,
            'latest': (ICSD_code in uniques_list)
        }
    )
    
    # # felix input dataset
    # Path(os.path.join(felix_input_dataset_loc, str(ICSD_code))).mkdir(parents=True, exist_ok=True)
    # # .cif
    # with open(os.path.join(felix_input_dataset_loc, str(ICSD_code), "felix.cif"), "w", encoding="utf8") as dest_file:
    #     dest_file.write(cif_text)
    # # .hkl
    # with open(os.path.join(felix_input_dataset_loc, str(ICSD_code), "felix.hkl"), "w", encoding="utf8") as dest_file:
    #     dest_file.write(hkl_content)
    # # .inp
    # if cell_length < 4:
    #     IMinReflectionPool = 250
    # elif cell_length > 14:
    #     IMinReflectionPool = 2500
    # else:
    #     IMinReflectionPool = (225 * cell_length) - 650
    
    # new_inp_content = re.sub("{{IMinReflectionPool}}", f"{int(IMinReflectionPool)}", inp_content)
    # new_inp_content = re.sub("{{IMinStrongBeams}}", f"{int(IMinReflectionPool / 4)}", new_inp_content)
    # new_inp_content = re.sub("{{ROuterConvergenceAngle}}", f"{(7.266 / cell_length):.5f}", new_inp_content)
    # with open(os.path.join(felix_input_dataset_loc, str(ICSD_code), "felix.inp"), "w", encoding="utf8") as dest_file:
    #     dest_file.write(new_inp_content)
        
keyinfo_df = pd.DataFrame(keyinfo)

In [68]:
keyinfo_df[keyinfo_df["latest"] == True]

,ICSD_code,chemical_formula_sum,cell_length,space_group,date,latest
0,101191,O35 Sn10.32 Ti8.39 Zn6.14,14.87930,F -4 3 m,2021-02-01,True
1,101256,H6.44 Ca3 Fe1.96 Ir1.43 O12,12.53910,I -4 3 d,2021-02-01,True
2,101275,I14 Sr1 W6,14.51923,P n -3 Z,2021-02-01,True
3,101277,I14 Mg1 W6,14.19333,P n -3 Z,2021-02-01,True
4,101279,Ca1 I14 W6,14.38351,P n -3 Z,2021-02-01,True
...,...,...,...,...,...,...
21596,99985,Co0.4 O6.8 Sm1.6 Ti2,10.19700,F d -3 m Z,2005-04-01,True
21597,99986,Ni0.11 O6.945 Sm1.89 Ti2,10.21900,F d -3 m Z,2005-04-01,True
21598,99987,Ni0.22 O6.89 Sm1.78 Ti2,10.21400,F d -3 m Z,2005-04-01,True
21599,99988,Ni0.3 O6.85 Sm1.7 Ti2,10.21300,F d -3 m Z,2005-04-01,True


In [34]:
keyinfo_df = pd.read_csv("keyinfo.csv", index_col="index")

def time(cell_length):
    x1 = 4
    y1 = 250
    x2 = 16
    y2 = 2400
    
    if cell_length < x1:
        beams = y1
    elif cell_length > x2:
        beams = y2
    else:
        beams = (y2-y1)/(x2-x1)*(cell_length-x1)+y1
    
    return 0.000000000001 * beams**4

keyinfo_df["time"] = keyinfo_df.apply(lambda row: time(row.cell_length), axis=1)
print(keyinfo_df["time"].sum() / 60)

1678.0410329732724


In [69]:
keyinfo_df.to_csv("keyinfo.csv", index_label="index")
keyinfo_df

,ICSD_code,chemical_formula_sum,cell_length,space_group,date,latest
0,101191,O35 Sn10.32 Ti8.39 Zn6.14,14.87930,F -4 3 m,2021-02-01,True
1,101256,H6.44 Ca3 Fe1.96 Ir1.43 O12,12.53910,I -4 3 d,2021-02-01,True
2,101275,I14 Sr1 W6,14.51923,P n -3 Z,2021-02-01,True
3,101277,I14 Mg1 W6,14.19333,P n -3 Z,2021-02-01,True
4,101279,Ca1 I14 W6,14.38351,P n -3 Z,2021-02-01,True
...,...,...,...,...,...,...
21596,99985,Co0.4 O6.8 Sm1.6 Ti2,10.19700,F d -3 m Z,2005-04-01,True
21597,99986,Ni0.11 O6.945 Sm1.89 Ti2,10.21900,F d -3 m Z,2005-04-01,True
21598,99987,Ni0.22 O6.89 Sm1.78 Ti2,10.21400,F d -3 m Z,2005-04-01,True
21599,99988,Ni0.3 O6.85 Sm1.7 Ti2,10.21300,F d -3 m Z,2005-04-01,True
